In [1]:
import re
import pandas as pd
import requests
import pdfplumber

In [41]:
arquivo = pdfplumber.open("arquivo/Extrato Banco.pdf")

In [34]:
with pdfplumber.open("arquivo/Extrato Agrupado Simplificado.pdf") as pdf:
    a = pdf.pages[0]
    a.chars[0]

In [35]:
b = a.extract_text().split("\n")

In [36]:
serie = pd.Series(b)

In [37]:
contratos = []
aluguel = []

In [38]:
for x in range (0, len(serie)):
    linha = serie.loc[x]
    primeira_palavra = linha.split(" ")[0]
    
    if primeira_palavra == "Emitido":
        data_emicao = linha.split(" ")[2]
        
    if primeira_palavra == "Contrato":
        contratos.append(linha[len("Contrato "):])
        
    if primeira_palavra == "Aluguel":
        aluguel.append(float((linha[len("Aluguel - 01/12/2020 até 31/12/2020 "):].replace(",", "."))))

In [39]:
aluguel

[368.3, 368.3, 429.68, 437.77]

In [40]:
dir(pdfplumber)

['PDF',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'container',
 'convert',
 'load',
 'open',
 'page',
 'pdf',
 'pdfminer',
 'set_debug',
 'sys',
 'table',
 'utils']